In [94]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from itertools import combinations

from statsmodels.tsa.stattools import adfuller

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import scale

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoCV

from scipy.stats import ttest_1samp

In [86]:
data_path = 'data/'

timeframe = 'monthly'
stat = '_stat'

if timeframe == 'quarterly':
    timescale = 4
elif timeframe == 'monthly':
    timescale = 1

In [87]:
total = pd.read_csv(data_path + f'total_{timeframe}{stat}.csv', delimiter=',', header=0, encoding='latin-1')
total['date'] = pd.to_datetime(total['date'])
total

,country,date,co2_m_stat,pop_m_stat,gdp_m_stat,co2_m_cap_stat,gdp_m_cap_stat
0,Austria,2001-01-01,-0.019138,7.874564e-04,-0.007274,0.100990,-0.008233
1,Austria,2001-02-01,0.039611,-6.066812e-04,-0.000168,-0.072598,-0.000521
2,Austria,2001-03-01,0.022160,-1.244510e-07,-0.000168,-0.065585,-0.000521
3,Austria,2001-04-01,0.119718,-1.243633e-07,-0.000168,-0.119950,-0.000521
4,Austria,2001-05-01,0.024326,3.718831e-05,0.001218,-0.251400,0.000828
...,...,...,...,...,...,...,...
4327,Switzerland,2019-08-01,0.027670,-2.316558e-04,0.001645,-0.016895,0.001240
4328,Switzerland,2019-09-01,-0.000445,-1.641248e-07,0.001642,0.209228,0.001237
4329,Switzerland,2019-10-01,0.008483,-1.639919e-07,0.001640,0.102677,0.001235
4330,Switzerland,2019-11-01,-0.004228,-1.638592e-07,0.001637,0.160760,0.001232


In [88]:
list(combinations(total.columns[2:], 2))[0]

('co2_m_stat', 'pop_m_stat')

In [89]:
treatment_country = 'France'
target_var = f'co2_{timeframe}{stat}'
impl_year = 2014

donor_countries = ['Austria', 'Belgium', 'Bulgaria', 
#                    'Cyprus', 
                   'Croatia', 'Czech Republic',
                   'Germany', 'Greece', 'Hungary', 'Italy', 'Lithuania', 'Netherlands', 
                   'Romania', 'Slovakia', 'Spain']

incl_years = pd.to_datetime(total['date']).dt.year.unique()
print(list(incl_years))
incl_countries = [treatment_country] + donor_countries
incl_countries.sort()
print(len(incl_countries))
print(incl_countries)

[2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019]
15
['Austria', 'Belgium', 'Bulgaria', 'Croatia', 'Czech Republic', 'France', 'Germany', 'Greece', 'Hungary', 'Italy', 'Lithuania', 'Netherlands', 'Romania', 'Slovakia', 'Spain']


In [90]:
cov_list = []
n_reg_list = []

n_pre_int = []
n_post_int = []

lasso_n_reg_list = []
lasso_R2_list = []

att_mean_list = []
att_sd_list = []
att_p_list = []
att_sign_list = []

for comb in [1]:
    for cov in list(combinations(total.columns[2:], comb)):
        cov_list.append(cov)
        cov = list(cov)

        # pivot donor data
        donors = total.copy()
        donors = donors[donors['country'].isin(donor_countries)].reset_index(drop=True)
        donors = donors.pivot(index='date', columns=['country'], values=cov)
        donors.columns = donors.columns.to_flat_index()
        donors.columns = [str(col_name[1]) + ' '+ str(col_name[0]) for col_name in donors.columns]
        donors = donors.reindex(sorted(donors.columns), axis=1)
        donors = donors.dropna(axis=1)
        donors

        # drop nonstationary
        for col in donors.columns:
            donor_series = donors[col]
            adf_test = adfuller(donor_series)
            if adf_test[1] < 0.05:
        #         print(f'{col}: Stationary')
                pass
            if adf_test[1] >= 0.05:
    #             print(f'{col}: Non-stationary ({adf_test[1]})')
    #             donors = donors.drop(col, axis=1)
                pass

        n_reg_list.append(donors.shape[1])

        # Select X, y
        X_cols = list(donors.columns)
        X_orig = donors[X_cols]

        y = np.array(total[total['country']==treatment_country][target_var]).reshape(-1,1)
        X = np.array(donors[X_cols])

        ### Sandardization of data ###
        PredictorScaler = StandardScaler()
        TargetVarScaler = StandardScaler()

        PredictorScalerFit = PredictorScaler.fit(X)
        TargetVarScalerFit = TargetVarScaler.fit(y)

        X = PredictorScalerFit.transform(X)
        y = TargetVarScalerFit.transform(y)

        # Split the data into training and testing set
        ts = int((2019 - impl_year + 1)*(12/timescale))
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=ts, random_state=42)

        n_pre_int.append(len(y_train))
        n_post_int.append(len(y_test))

        # Fit LASSO
        tscv = TimeSeriesSplit(n_splits=round(len(X_train)/5))
        model = LassoCV(
                        alphas=np.arange(0.01, 1, 0.001), 
        #                 alphas=[0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000], 
                        cv=tscv,
                        fit_intercept=True,
                        max_iter=100000,
                        tol=0.00001
                       )
        model.fit(X_train, y_train.ravel())

        lasso_n_reg_list.append(len(list(model.coef_)))
        lasso_R2_list.append(model.score(X_train, y_train))

        act = TargetVarScalerFit.inverse_transform(y_test)
        pred = TargetVarScalerFit.inverse_transform(model.predict(X_test).reshape(-1, 1))
        diff = act - pred

        res = ttest_1samp(diff, popmean=0)
        att_mean_list.append(np.mean(diff))
        att_sd_list.append(np.std(diff))
        att_p_list.append(res[1][0])
        if res[1][0] < 0.05:
            att_sign_list.append(1)
        else:
            att_sign_list.append(0)

results = pd.DataFrame(list(zip(cov_list, n_reg_list, n_pre_int, n_post_int, lasso_n_reg_list, lasso_R2_list, 
                          att_mean_list, att_sd_list, att_p_list, att_sign_list)), 
                  columns =['covariates', 'n_regressors', 'n_pre_intervention', 'n_post_intervention',
                           'n_cov_lasso', 'lasso_R2', 'att_mean', 'att_sd', 'att_p_value', 'att_significant'])    
results

,covariates,n_regressors,n_pre_intervention,n_post_intervention,n_cov_lasso,lasso_R2,att_mean,att_sd,att_p_value,att_significant
0,"(co2_m_stat,)",14,156,72,14,0.747986,-0.000681,0.032431,0.860160,0
1,"(pop_m_stat,)",14,156,72,14,0.000000,0.008447,0.063861,0.268788,0
2,"(gdp_m_stat,)",14,156,72,14,0.000000,0.008447,0.063861,0.268788,0
3,"(co2_m_cap_stat,)",14,156,72,14,0.000000,0.008447,0.063861,0.268788,0
4,"(gdp_m_cap_stat,)",14,156,72,14,0.000000,0.008447,0.063861,0.268788,0


In [95]:
corr_matrix = total.corr()
sns.heatmap(corr_matrix, annot=True)
plt.show()